### NEW LIBRARY

In [2]:
from pyart.io import uf
from gzip import open as gzip_open


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
uf_filename = "./data/1105/ppi/olympex_NPOL1_20151105_161254_uf.gz"
with gzip_open(uf_filename, 'rb') as unzipped_file:
    print(unzipped_file)
    radar = uf.read_uf(
        unzipped_file,
        file_field_names=True
    )
radar

<gzip _io.BufferedReader name='./data/1105/ppi/olympex_NPOL1_20151105_161254_uf.gz' 0x7ff36bd8df60>


In [4]:
sweep_start_ray_idx = radar.sweep_start_ray_index['data'][:]
sweep_end_ray_idx = radar.sweep_end_ray_index['data'][:]

print(sweep_start_ray_idx)
print(sweep_end_ray_idx)

[  0 360 720]
[ 359  719 1079]


In [5]:
cz = (radar.fields['CZ']['data'][:])
dr = (radar.fields['DR']['data'][:])
rh = (radar.fields['RH']['data'][:])

In [7]:
type(cz)

numpy.ma.core.MaskedArray

In [14]:
cz.mask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [20]:
# mask when true, it indicates an invalid data.
# mask when false, it indicates an valid data.
cz[~cz.mask]

masked_array(data=[],
             mask=[],
       fill_value=1e+20,
            dtype=float64)

In [21]:
cz.data

array([[-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68],
       [-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68],
       [-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68],
       ...,
       [-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68],
       [-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68],
       [-327.68, -327.68, -327.68, ..., -327.68, -327.68, -327.68]])

In [23]:
import numpy as np

In [24]:
np.mean(cz.data)

-327.68000000000023

VISAGE REFERENCE

In [25]:
cz = (radar.fields['CZ']['data'][:])

In [33]:
rh

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float64)

In [35]:
gate_latitude = radar.gate_latitude['data'][:]
gate_longitude = radar.gate_longitude['data'][:]
gate_altitude = radar.gate_altitude['data'][:]

In [38]:
gate_altitude

array([[ 157.,  158.,  159., ..., 2366., 2369., 2372.],
       [ 157.,  158.,  159., ..., 2366., 2369., 2372.],
       [ 157.,  158.,  159., ..., 2366., 2369., 2372.],
       ...,
       [ 159.,  164.,  170., ..., 7067., 7075., 7082.],
       [ 159.,  164.,  170., ..., 7067., 7075., 7082.],
       [ 159.,  164.,  170., ..., 7067., 7075., 7082.]])

In [40]:
from datetime import datetime

full_time = radar.time['units']\
    .replace('since', '')\
    .replace('seconds', '')
full_time = full_time.replace(' ', '')
full_time = full_time.replace('T', '')
full_time = datetime.strptime(full_time, '%Y-%m-%d%H:%M:%SZ')

In [41]:
full_time

datetime.datetime(2015, 11, 5, 16, 12, 56)

In [48]:
from datetime import timedelta
from numpy import ma

CZ = cz
DR = dr
RH = rh

In [54]:
row_dict = {}

In [58]:
for ii in range(0, radar.nsweeps):
    # Calculate the start and end time for this sweep
    idx0 = sweep_start_ray_idx[ii]
    idx1 = sweep_end_ray_idx[ii]

    # ray by ray; 226 rays per sweep
    for ray in range(idx0, idx1 + 1):
        tmp_cz = CZ[ray, :]
        tmp_dr = DR[ray, :]
        tmp_rh = RH[ray, :]
        
        tmp_time_ray = (full_time + timedelta(
            seconds=float(radar.time['data'][ray])
        )).strftime('%Y-%m-%dT%H:%M:%SZ')
        tmp_gate_lat = gate_latitude[ray, :]
        tmp_gate_lon = gate_longitude[ray, :]
        tmp_gate_alt = gate_altitude[ray, :]

        # check CZ values gate by gate; 1081 gates per ray
        for gate in range(0, len(tmp_cz)):
            # if tmp_cz[gate] is not ma.masked:
                row_dict = {
                    'timestamp': tmp_time_ray,
                    'lat': round(float(tmp_gate_lat[gate]), 4),
                    'lon': round(float(tmp_gate_lon[gate]), 4),
                    'height': float(tmp_gate_alt[gate]),
                    'CZ': float(tmp_cz[gate]),
                    'DR': float(tmp_dr[gate]),
                    'RH': float(tmp_rh[gate]),

                }

/tmp/ipykernel_1901/2095481122.py:27: UserWarning: Warning: converting a masked element to nan.
  'CZ': float(tmp_cz[gate]),
/tmp/ipykernel_1901/2095481122.py:28: UserWarning: Warning: converting a masked element to nan.
  'DR': float(tmp_dr[gate]),
/tmp/ipykernel_1901/2095481122.py:29: UserWarning: Warning: converting a masked element to nan.
  'RH': float(tmp_rh[gate]),


In [59]:
row_dict

{'timestamp': '2015-11-05T16:14:09Z',
 'lat': 48.4896,
 'lon': -124.242,
 'height': 7082.0,
 'CZ': nan,
 'DR': nan,
 'RH': nan}

In [80]:
# Update data with new API updated
# OLYMPEX
# (6) 
#Ground radar NPOL (gpmnpololyx2)  
#collection = GroundRadar
#campaign = olympex
#instrument = NPOL
#scan type = RHI

from pyart.io import uf
from datetime import timedelta
from datetime import datetime
from numpy import ma
from gzip import open as gzip_open

class Reader():
    """Reader that reads all data from a set of UF Radar Files.
    """
    def init(self):
        pass

    def _read_radar(self):
        """Read the radar file and return some important values.

        :return: The radar data including some important values.
        :rtype: Tuple
        """
        uf_filename = "./data/1203/rhi_a/olympex_NPOL1_20151203_000005_rhi_00-20.uf.gz"
        if uf_filename.endswith('.gz'):
            with gzip_open(uf_filename, 'rb') as unzipped_file:
                radar = uf.read_uf(
                    unzipped_file,
                    file_field_names=True
                )
        else:
            radar = uf.read_uf(
                uf_filename,
                file_field_names=True
            )
        sweep_start_ray_idx = radar.sweep_start_ray_index['data'][:]
        sweep_end_ray_idx = radar.sweep_end_ray_index['data'][:]

        cz = (radar.fields['CZ']['data'][:])
        dr = (radar.fields['DR']['data'][:])
        rh = (radar.fields['RH']['data'][:])
        
        fh = (radar.fields['FH']['data'][:])
        dm = (radar.fields['DM']['data'][:])


        # Lucy Wang added the following command lines on July 23, 2018
        gate_latitude = radar.gate_latitude['data'][:]
        gate_longitude = radar.gate_longitude['data'][:]
        gate_altitude = radar.gate_altitude['data'][:]
        # -------------------------------------------------------------

        # Get time in %Y-%m-%d%H:%M:%SZ format
        full_time = radar.time['units']\
            .replace('since', '')\
            .replace('seconds', '')
        full_time = full_time.replace(' ', '')
        full_time = full_time.replace('T', '')
        full_time = datetime.strptime(full_time, '%Y-%m-%d%H:%M:%SZ')

        return (
            sweep_start_ray_idx,
            sweep_end_ray_idx,
            cz,
            dr,
            rh,
            fh,
            dm,
            gate_latitude,
            gate_longitude,
            gate_altitude,
            full_time,
            radar,
        )

    def read_data(self):
        """Generator function that generates each datum from the radar file
        sequentially.
        """
        index = 0
        (
            sweep_start_ray_idx,
            sweep_end_ray_idx,
            CZ,
            DR,
            RH,
            FH,
            DM,
            gate_latitude,
            gate_longitude,
            gate_altitude,
            full_time,
            radar
        ) = self._read_radar()

        # sweep by sweep; 20 sweeps per rhi_a file (over ocean)
        for ii in range(0, radar.nsweeps):
            # Calculate the start and end time for this sweep
            idx0 = sweep_start_ray_idx[ii]
            idx1 = sweep_end_ray_idx[ii]

            # ray by ray; 226 rays per sweep
            for ray in range(idx0, idx1 + 1):
                tmp_cz = CZ[ray, :]
                tmp_dr = DR[ray, :]
                tmp_rh = RH[ray, :]
                tmp_fh = FH[ray, :]
                tmp_dm = DM[ray, :]
                tmp_time_ray = (full_time + timedelta(
                    seconds=float(radar.time['data'][ray])
                )).strftime('%Y-%m-%dT%H:%M:%SZ')
                tmp_gate_lat = gate_latitude[ray, :]
                tmp_gate_lon = gate_longitude[ray, :]
                tmp_gate_alt = gate_altitude[ray, :]

                # check CZ values gate by gate; 1081 gates per ray
                for gate in range(0, len(tmp_cz)):
                    if tmp_cz[gate] is not ma.masked:
                        row_dict = {
                            'timestamp': tmp_time_ray,
                            'lat': round(float(tmp_gate_lat[gate]), 4),
                            'lon': round(float(tmp_gate_lon[gate]), 4),
                            'height': float(tmp_gate_alt[gate]),
                            'CZ': float(tmp_cz[gate]),
                            'DR': float(tmp_dr[gate]),
                            'RH': float(tmp_rh[gate]),
                            'FH': float(tmp_fh[gate]),
                            'DM': float(tmp_dm[gate]),
                        }
                        yield row_dict
                        index += 1


In [81]:
rd = Reader()
data = rd.read_data()

In [82]:
data

<generator object Reader.read_data at 0x7ff36bba7ed0>

In [83]:
print(next(data))

{'timestamp': '2015-12-03T00:00:07Z', 'lat': 47.2709, 'lon': -124.2165, 'height': 156.0, 'CZ': 25.93, 'DR': 1.22, 'RH': 0.84, 'FH': 2.0, 'DM': 1.85}


Generators

In [70]:
def fun(x):
    n = 0
    while n < x:
        yield n
        n += 1
z = fun(10)
next(z)
next(z)

1

In [71]:
next(z)

2